# Attachment Protocol Example - Alice

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. This protocol allows you to attach a file to a message.
Run this along side [Attachment Protocol Bob](http://localhost:8889/notebooks/attachment.ipynb)

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


####  Intialise a controller 

In [2]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

### Confirm that an active connection exists

In [3]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'invitation_key': '89jbF3LWuCeUdE8TJzuMP5iVJaWeyGCKv2rfbdn2ZMBU', 'connection_id': '459851bf-6d7b-4ffa-8811-b4209d7543ef', 'request_id': 'f205bccf-1a0e-4a67-997b-97e4ea534b89', 'their_label': 'Bob', 'initiator': 'external', 'invitation_mode': 'once', 'accept': 'manual', 'state': 'active', 'updated_at': '2020-09-10 12:08:40.257038Z', 'created_at': '2020-09-10 12:05:20.030535Z', 'their_did': '6QZAPijnyMrQgGvnpb9x9m', 'routing_state': 'none', 'my_did': '5UZRMryktBo8Nhzz2q6p2G'}]
Connection : {'invitation_key': '89jbF3LWuCeUdE8TJzuMP5iVJaWeyGCKv2rfbdn2ZMBU', 'connection_id': '459851bf-6d7b-4ffa-8811-b4209d7543ef', 'request_id': 'f205bccf-1a0e-4a67-997b-97e4ea534b89', 'their_label': 'Bob', 'initiator': 'external', 'invitation_mode': 'once', 'accept': 'manual', 'state': 'active', 'updated_at': '2020-09-10 12:08:40.257038Z', 'created_at': '2020-09-10 12:05:20.030535Z', 'their_did': '6QZAPijnyMrQgGvnpb9x9m', 'routing_state': 'none', 'my_did': '5UZRMryktBo8Nhzz2q6p2G'}
Active Conne

### Instantiate Listener for the Attachment topic

This is emitted using PyPubSub when the controller receives a atatchment webhook from the agent. This happens everytime the agent receives an attachment.

In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

attach_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([attach_listener], defaults=True)

### Send an attachment over DIDcomm to Bob

In this instance a text file is sent, this will be received and saved by the handler in the [Bob notebook](http://localhost:8889/notebooks/attachment.ipynb).

In [15]:
from aiohttp import FormData
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': '10f7e2c2-e4bf-47c5-a267-387fb11f0b01'}



# End of Tutorial

Be sure to terminate the controller so you can run another tutoria

In [8]:
response = await om_controller.terminate()
print(response)


None
